In [152]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
from selenium import webdriver
import requests
import pymongo
from IPython.display import Image
import shutil
from splinter.exceptions import ElementDoesNotExist
from selenium.webdriver.common.keys import Keys

In [153]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

which: no chromedriver in (/c/Users/sinancengiz/Anaconda3/envs/PythonData:/c/Users/sinancengiz/Anaconda3/envs/PythonData/Library/mingw-w64/bin:/c/Users/sinancengiz/Anaconda3/envs/PythonData/Library/usr/bin:/c/Users/sinancengiz/Anaconda3/envs/PythonData/Library/bin:/c/Users/sinancengiz/Anaconda3/envs/PythonData/Scripts:/c/Users/sinancengiz/Anaconda3/envs/PythonData/bin:/c/Users/sinancengiz/bin:/mingw64/bin:/usr/local/bin:/usr/bin:/usr/bin:/mingw64/bin:/usr/bin:/c/Users/sinancengiz/bin:/c/ProgramData/Oracle/Java/javapath:/c/Program Files (x86)/Intel/iCLS Client:/c/Program Files/Intel/iCLS Client:/c/WINDOWS/system32:/c/WINDOWS:/c/WINDOWS/System32/Wbem:/c/WINDOWS/System32/WindowsPowerShell/v1.0:/c/Program Files (x86)/Intel/Intel(R) Management Engine Components/DAL:/c/Program Files/Intel/Intel(R) Management Engine Components/DAL:/c/Program Files (x86)/Intel/Intel(R) Management Engine Components/IPT:/c/Program Files/Intel/Intel(R) Management Engine Components/IPT:/cmd:/c/Program Files (x86)/

In [154]:
executable_path = {'executable_path': 'chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [155]:
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

In [156]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

header = soup.find_all('div', class_='content_title')
paragraf = soup.find_all('div', class_='article_teaser_body')


print(paragraf[0].text)

IndexError: list index out of range

In [ ]:
news_title = header[0].text
news_p = paragraf[0].text

In [ ]:
news_title

In [ ]:
news_p

In [178]:
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [187]:
image = soup.find_all('div', class_='img')
url = image[0].img["src"]
url

'/spaceimages/images/wallpaper/PIA22400-640x350.jpg'

In [191]:
featured_image_url = ['https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars' + url]
print(featured_image_url)

['https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars/spaceimages/images/wallpaper/PIA22400-640x350.jpg']


In [190]:
response = requests.get(featured_image_url, stream=True)
with open('img.png', 'wb') as out_file:
    shutil.copyfileobj(response.raw, out_file)

InvalidSchema: No connection adapters were found for '['https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars/spaceimages/images/wallpaper/PIA22400-640x350.jpg']'

In [161]:
url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
results = soup.find_all('div', class_='js-tweet-text-container')

In [162]:
mars_weather = results[0].p.text
print(mars_weather)

Sol 2060 (May 23, 2018), Sunny, high 4C/39F, low -73C/-99F, pressure at 7.43 hPa, daylight 05:20-17:20


In [26]:
url = 'http://space-facts.com/mars/'

tables = pd.read_html(url)
tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.42 x 10^23 kg (10.7% Earth)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.52 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                  -153 to 20 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers]

In [27]:
df = tables[0]
df.columns = ['Planet Data Type', 'Value']
df.set_index('Planet Data Type', inplace=True)
df.head()

,Value
Planet Data Type,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"


In [28]:
html_table = df.to_html(index=False)
html_table.replace('\n', '')
df.to_html('table.html')

In [29]:
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [30]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [31]:
# Define database and collection
db = client.craigslist_db
collection = db.items

In [32]:
# URL of page to be scraped
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')


In [33]:
mars_items = soup.find_all(class_="item")
url_list = []
titles =[]
hemisphere_image_urls =[]
for item in mars_items:
    title = item.div.h3.text
    titles.append(title)
    print(title)
    link = item.a["href"]
    image_link = 'https://astrogeology.usgs.gov' + link
    url_list.append(image_link)
    print(image_link)
    
    post = {
        "title": title,
        "img_url": image_link,
    }
    hemisphere_image_urls.append(post)


Cerberus Hemisphere Enhanced
https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced
Schiaparelli Hemisphere Enhanced
https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced
Syrtis Major Hemisphere Enhanced
https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced
Valles Marineris Hemisphere Enhanced
https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced


In [40]:
all_data_from_scrape = {
    "list_of_hemisphere_image_urls": hemisphere_image_urls,
    "Planet Data": df,
    "Mars_weather": mars_weather,
    }

all_data_from_scrape    

{'Mars_weather': 'Sol 2060 (May 23, 2018), Sunny, high 4C/39F, low -73C/-99F, pressure at 7.43 hPa, daylight 05:20-17:20',
 'Planet Data':                                               Value
 Planet Data Type                                   
 Equatorial Diameter:                       6,792 km
 Polar Diameter:                            6,752 km
 Mass:                 6.42 x 10^23 kg (10.7% Earth)
 Moons:                          2 (Phobos & Deimos)
 Orbit Distance:            227,943,824 km (1.52 AU)
 Orbit Period:                  687 days (1.9 years)
 Surface Temperature:                  -153 to 20 °C
 First Record:                     2nd millennium BC
 Recorded By:                   Egyptian astronomers,
 'list_of_hemisphere_image_urls': [{'img_url': 'https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced',
   'title': 'Cerberus Hemisphere Enhanced'},
  {'img_url': 'https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced',
   'title': 'Schiapa

In [ ]:
book_url_list = ['https://astrogeology.usgs.gov' + url for url in url_list]

titles_and_urls = zip(titles, book_url_list)

book_url_list

In [ ]:
# book_url_list = ['https://astrogeology.usgs.gov' + url for url in url_list]

# book_url_list[0]
# url = book_url_list[0]
# browser.click_link_by_href(url)

In [ ]:
print(titles)
print(url_list)